<span STYLE="font-size:150%"> 
    Segment polygonal regions from microCT scans
</span>

Docker image: gnasello/slicer-env:2023-10-10 \
Latest update: 11 October 2023

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

# Load libraries

In [1]:
import pyslicer as ps
import slicer
from pathlib import Path
import pandas as pd

# Volume input

## Load `.nrrd` file into 3D Slicer

Write the path of the `.nrrd` file and load it to Slicer

In [ ]:
# this cell is tagged 'parameters'
volume_file = 'microCT_volume/microCT_volume.nrrd'
output_polygon_dir_path = 'segmented_volumes/polygonDefect'
file_nuclei_segmentation = 'segmented_volumes/Bone.seg.nrrd'
directory_notebook = Path().parent.absolute()
sample_name = directory_notebook.stem
camera_view_file = 'segmented_volumes/camera_view.csv'

In [2]:
path = Path(volume_file)

# Remove image numbering _0000, _0001 ...
filename_output = path.stem[:-4]

In [3]:
masterVolumeNode = slicer.util.loadNodeFromFile(volume_file)

Print spacing

In [4]:
## mm
masterVolumeNode.GetSpacing()

(0.007716500000000002, 0.007716500000000002, 0.007716500000000002)

Make ```segmented_volumes``` folder

In [6]:
output_polygonDefect_path = Path(output_polygon_dir_path)

output_polygonDefect_path.mkdir(parents=True, exist_ok=True)

Get norebook directory name

In [7]:
sample_name

'AS-L_4'

## Adjust window/level (brightness/contrast)

Adjust the image window/level from the GUI. It is enough to select a region where the signal is present ([youtube](https://slicer.readthedocs.io/en/latest/user_guide/user_interface.html#adjusting-image-window-level)). 

After, get the Window and Level properties from the displayNode associate to the VolumeNode and apply them programmatically anytime you run again the same image.

For more information on the relationship between window/level and brightness/contrast, see [Window and Level Contrast Enhancement](http://fisica.ciens.ucv.ve/curs/dipcourse/html/one-oper/window-level/front-page.html)

In [8]:
displayNode = masterVolumeNode.GetDisplayNode()
print('displayNode.SetWindow(' + str(displayNode.GetWindow()) + ')')
print('displayNode.SetLevel(' + str(displayNode.GetLevel()) + ')')

displayNode.SetWindow(17233.0)
displayNode.SetLevel(17305.5)


In [9]:
# displayNode = masterVolumeNode.GetDisplayNode()
# displayNode.AutoWindowLevelOff()
# displayNode.SetWindow(12415.0)
# displayNode.SetLevel(11271.5)

# Create segmentationNode

## Create segmentation-related nodes

Create segmentation node

In [10]:
segmentationNode = ps.segmentation.segmentationNode(name='Segmentation')

Create temporary segment editor to get access to effects

In [11]:
segmentEditorWidget, segmentEditorNode = ps.segmentation.segmentEditorWidget(segmentationNode = segmentationNode, 
                                                                             masterVolumeNode = masterVolumeNode)

## Load bone segmentation to segmentation node

Load segmentation from .nrrd file as labelmap node slicer util module to load from File [github](https://github.com/Slicer/Slicer/blob/master/Base/Python/slicer/util.py#L341-L344)

In [12]:
# Load segmentation from .seg.nrrd file (includes segment names and colors)

bone_labelmap = slicer.util.loadNodeFromFile(file_nuclei_segmentation, 'VolumeFile', properties={'labelmap':True})
# segmentationNode = slicer.util.loadSegmentation(file_nuclei_segmentation)

Import bone labelmapNode to segmentationNode

In [13]:
slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(bone_labelmap, segmentationNode)

True

Rename segment of the segmentation node 

In [14]:
segmentation = segmentationNode.GetSegmentation()
segment = segmentation.GetNthSegment(0)
segment.SetName("Bone")

Delete bone labelmapNode

In [15]:
slicer.mrmlScene.RemoveNode(bone_labelmap)
# masterVolumeNode.SetDisplayVisibility(1)

## Load defect cylinders in segment node

Load `.stl` files as models to Slicer

In [16]:
cylinder_files = [output_polygonDefect_path / 'innerDefectPolygon.vtk',
                  output_polygonDefect_path / 'defectPolygon.vtk'
                 ]

color_list = [
         (230/255, 75/255, 53/255),
         (77/255, 187/255, 213/255),
        ]

cylinderModel_nodes = []

for i, file in enumerate(cylinder_files):
    
    color = color_list[i]
    
    model = slicer.util.loadModel(file)
    model.SetName('Ingrowth_' + file.stem)
    cylinderModel_nodes.append(model)

    slicer.modules.segmentations.logic().ImportModelToSegmentationNode(model, segmentationNode)

    modelDisplayNode = model.GetDisplayNode()
    modelDisplayNode.SetColor(color[0], color[1], color[2])
    modelDisplayNode.SetOpacity(0.4)

Set segment colors

In [17]:
segments_color = {'Bone':(0.9450980392156862, 0.8392156862745098, 0.5686274509803921), # "Bone" color in Slicer
                  'Ingrowth_innerDefectPolygon':(230/255, 75/255, 53/255),
                  'Ingrowth_defectPolygon':(77/255, 187/255, 213/255)
                  }

ps.segmentation.set_segments_color(segments_color, segmentationNode)

# Operation on segments

## Manual fix of the segmentation

Sometimes it might be necessary to remove speckles at the image boundaries. If so, use the `scissor` tool in the `Segment Editor` before proceeding with the rest of the script. 

## Intersect cylinder segments with bone segment

In [18]:
ps.segmentation.logical_intersect('Ingrowth_innerDefectPolygon', 'Bone', segmentationNode, segmentEditorNode, segmentEditorWidget)
ps.segmentation.logical_intersect('Ingrowth_defectPolygon', 'Bone', segmentationNode, segmentEditorNode, segmentEditorWidget)

# Compute bone ingrowth volumes

Delete `Bone` segment before computing volumes

In [19]:
segmentName = 'Bone'
segmentId = segmentationNode.GetSegmentation().GetSegmentIdBySegmentName(segmentName)
segmentationNode.GetSegmentation().RemoveSegment('1')

Compute segment statistics

In [20]:
stats = ps.segmentation.segment_statistics(segmentationNode)

Store volume of each segment

In [21]:
segment_names = []
volumes_ingrowth_mm3 = []

# Display volume of each segment
for segmentId in stats["SegmentIDs"]:
    
    volume_mm3 = stats[segmentId,"LabelmapSegmentStatisticsPlugin.volume_mm3"]
    volumes_ingrowth_mm3.append(volume_mm3)
    
    segmentName = segmentationNode.GetSegmentation().GetSegment(segmentId).GetName()
    segment_names.append(segmentName)

data_dict = {'segmentName': segment_names, 'volume_mm3':volumes_ingrowth_mm3}
df = pd.DataFrame(data_dict)
df

,segmentName,volume_mm3
0,Ingrowth_innerDefectPolygon,0.000805
1,Ingrowth_defectPolygon,0.079025


Add sample name columns to DataFrame

In [22]:
df['sample'] = [sample_name] * len(df. index)
df

,segmentName,volume_mm3,sample
0,Ingrowth_innerDefectPolygon,0.000805,AS-L_4
1,Ingrowth_defectPolygon,0.079025,AS-L_4


In [23]:
outputfile = output_polygonDefect_path / (sample_name + '_volume_ingrowth.csv')
df.to_csv(outputfile, index=False)

# Export segments

## Export all segments to individual `.stl` files

Get closed surface representation of the segment, from [slicer scripting repository](https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html#export-nodes-warped-by-transform-sequence)

In [24]:
segmentationNode.CreateClosedSurfaceRepresentation()

True

In [25]:
slicer.modules.segmentations.logic().ExportSegmentsClosedSurfaceRepresentationToFiles(str(output_polygonDefect_path), segmentationNode)

True

# Visualize segmentations in 3D view

Get viewNode

In [26]:
view = slicer.app.layoutManager().threeDWidget(0).threeDView()
viewNode = view.mrmlViewNode()

# Switch off cube and labels
viewNode.SetAxisLabelsVisible(False)
viewNode.SetBoxVisible(False)

color = (28/255, 29/255, 36/255)
# Set view background to RGB color of choice
viewNode.SetBackgroundColor(color[0], color[1], color[2])
viewNode.SetBackgroundColor2(color[0], color[1], color[2])

# Set Orthographic rendering, which is required to show the ruler in a 3D view
viewNode.SetRenderMode(viewNode.Orthographic)

# Set thick and white ruler
viewNode.SetRulerType(2) # 2 - thick
viewNode.SetRulerColor(0) # 0 - white

# Get camera position
cameraNode = slicer.modules.cameras.logic().GetViewActiveCameraNode(viewNode)

## Get camera position in 3D view

Load `camera_view.csv` file

In [27]:
df_camera = pd.read_csv(camera_view_file)
df_camera

,position,viewUp,focalPoint,viewAngle,parallelScale
0,-19.969042,-0.148436,-4.723929,30.0,6.469837
1,-4.091685,-0.238192,-4.822962,NaN,NaN
2,10.066162,-0.959808,7.889958,NaN,NaN


## Set lighting in 3D view

In [28]:
slicer.util.selectModule('Lights')
moduleWidget = slicer.modules.lights.widgetRepresentation().self()

Set the Managed 3D view

In [29]:
moduleWidget.logic.addManagedView(viewNode)

Set Key light parameters

In [30]:
LightIntensity = 0.92
LightWarmth = 0.36
LightElevation = 47.45
LightAzimuth = 16.50

moduleWidget.logic.lightKit.SetKeyLightIntensity(LightIntensity)
moduleWidget.logic.lightKit.SetKeyLightWarmth(LightWarmth)
moduleWidget.logic.lightKit.SetKeyLightElevation(LightElevation)
moduleWidget.logic.lightKit.SetKeyLightAzimuth(LightAzimuth)

In [31]:
moduleWidget.updateWidgetFromLogic()

## Set camera position in 3D view

Choose pre-set values for camera position, focal point and view up direction

In [32]:
cameraNode.SetPosition(df_camera['position'])
cameraNode.SetViewAngle(df_camera['viewAngle'][0])
cameraNode.SetViewUp(df_camera['viewUp'])
cameraNode.SetFocalPoint(df_camera['focalPoint'])
cameraNode.SetParallelScale(df_camera['parallelScale'][0])